In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## ミニバッチ法のPyTorchによる実装
PyTorchでミニバッチ法を実装してみましょう。
基礎編で使った2次元データとMLPモデルを例に実験します。

In [ ]:

# 二次元ガウス分布と一様分布
def get_dataset_2():
    from numpy.random import default_rng
    rng = default_rng(seed=0)  # 今回はデータセットの乱数を固定させます。

    num_signal = 100  # 生成するシグナルイベントの数
    num_background = 1000  # 生成するバックグラウンドイベントの数

    # データ点の生成
    ## 平均(x1,x2) = (1.0, 0.0)、分散=1の２次元ガウス分布
    x_sig = rng.multivariate_normal(mean=[1.0, 0],
                                    cov=[[1, 0], [0, 1]],
                                    size=num_signal)
    t_sig = np.ones((num_signal, 1))  # Signalは1にラベリング

    ## (-5, +5)の一様分布
    x_bg = rng.uniform(low=-5, high=5, size=(num_background, 2))
    t_bg = np.zeros((num_background, 1))  # Backgroundは0にラベリング

    # 2つのラベルを持つ学習データを1つにまとめる
    x = np.concatenate([x_sig, x_bg])
    t = np.concatenate([t_sig, t_bg])

    # データをランダムに並び替える
    p = rng.permutation(len(x))
    x, t = x[p], t[p]

    return x, t


# ラベル t={0,1}を持つデータ点のプロット
def plot_datapoint(x, t):
    # シグナル/バックグラウンドの抽出
    xS = x[t[:, 0] == 1]  # シグナルのラベルだけを抽出
    xB = x[t[:, 0] == 0]  # バックグラウンドのラベルだけを抽出

    # プロット
    plt.scatter(xS[:, 0], xS[:, 1], label='Signal', c='red', s=10)  # シグナルをプロット
    plt.scatter(xB[:, 0], xB[:, 1], label='Background', c='blue', s=10)  # バックグラウンドをプロット
    plt.xlabel('x1')  # x軸ラベルの設定
    plt.ylabel('x2')  # y軸ラベルの設定
    plt.legend()  # legendの表示
    plt.show()


# prediction関数 の等高線プロット (fill)
def plot_prediction_torch(model, *args):
    from torch import from_numpy

    # 等高線を描くためのメッシュの生成
    x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
    x1 = x1.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x2 = x2.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x = np.array([x1, x2]).T
    x = from_numpy(x).float()

    #  関数predictionを使って入力xから出力yを計算し、等高線プロットを作成
    y = model(x, *args)
    y = y.detach().numpy()  # PyTroch Tensorからnumpy arrayへの変換
    cs = plt.tricontourf(x[:, 0], x[:, 1], y.flatten(), levels=10)
    plt.colorbar(cs)


In [ ]:
# データ点の取得 (データセットのサイズは1100です。)
x, t = get_dataset_2()

from torch import from_numpy
# numpy -> torch.tensor
x = from_numpy(x).float()
t = from_numpy(t).float()
print("データのサイズ", x.shape, t.shape)

まずは一度に全てのデータを使ってロスの計算を行うバッチ学習を試してみます。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import BCELoss
from torch.optim import SGD
import time

# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 順伝搬
    y_pred = model(x)

    # ロスの計算
    loss = loss_fn(y_pred, t)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}')

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)

次に1行ずつロスを計算するオンライン学習で学習を実行します。

In [ ]:
# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 1行ずつロスを計算し、重みを更新する。
    for xb, tb in zip(x, t):
        # 順伝搬
        y_pred = model(xb)

        # ロスの計算
        loss = loss_fn(y_pred, tb)

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()
        
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}')

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)

- オンライン学習では1ステップあたりの時間は短いですが、1100回重みの更新をする必要があるためトータルの時間はバッチ法よりも長くかかっています。
- 今回は全データを１回ずつ使用して(エポック数1で)学習したためバッチ法では学習が十分に進んでいません。エポック数を増やすことで学習をさらに進めることができます。

次にミニバッチ法を試してみましょう。バッチサイズは10としてみます。

In [ ]:
# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 10行ずつロスを計算し、重みを更新する。
    for i_batch in range(110):
        # 10行ずつデータを切り出す
        xb = x[10 * i_batch: 10 * (i_batch + 1)]
        tb = t[10 * i_batch: 10 * (i_batch + 1)]
 
        # 順伝搬
        y_pred = model(xb)

        # ロスの計算
        loss = loss_fn(y_pred, tb)

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()
        
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}')

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)

1回の重み更新で10イベントを処理するので、全体として110ステップ分処理することになりました。
振る舞いはバッチ学習とオンライン学習の中間くらいになっています。

今回はデータセットがシンプルだったため、オンライン学習でも問題なく学習が進みましたが、オンライン学習は学習が不安定になることが多いです。
そのため実際の深層学習モデル学習の際は、バッチサイズが32 ~ 2048程度のミニバッチ学習を使うことが多いです。

ミニバッチの処理をする際、毎度データをバッチに切り出すコードを書くのは大変なので、通常はDataLoaderというクラスを用いてこれを処理します。
上記のミニバッチのコードは以下のように書き換えることができます。

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch import tensor
ds = TensorDataset(x, t)
dataloader = DataLoader(ds, batch_size=10)  # バッチサイズとして10を指定します。

# モデルの定義
model = Sequential(
    Linear(in_features=2, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=64),  # ノード数が64の層を追加。
    ReLU(),  # 活性化関数はシグモイド関数。
    Linear(in_features=64, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)

# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01)

# トレーニング
for i_epoch in range(1):
    # 時間の測定
    start_time = time.time()

    # 10行ずつロスを計算し、重みを更新する。
    for xb, tb in dataloader:
        # 順伝搬
        y_pred = model(xb)

        # ロスの計算
        loss = loss_fn(y_pred, tb)

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()
        
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss}')

model.eval()

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction_torch(model)

## データ点をプロット
plot_datapoint(x, t)